In [1]:
#import
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn import metrics
import shap

In [2]:
#Reading dataset
df=pd.read_csv('DFT.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Cate_name          68 non-null     object 
 1   Cate_HOMO          68 non-null     float64
 2   Cate_LUMO          68 non-null     float64
 3   Cate_ESP_O1        68 non-null     float64
 4   Cate_ESP_H1        68 non-null     float64
 5   Cate_ESP_O2        68 non-null     float64
 6   Cate_ESP_H2        68 non-null     float64
 7   Cate_BL_ArO1       68 non-null     float64
 8   Cate_BL_O1H1       68 non-null     float64
 9   Cate_BL_ArO2       68 non-null     float64
 10  Cate_BL_O2H2       68 non-null     float64
 11  Cate_total_dipole  68 non-null     float64
 12  P_name             68 non-null     object 
 13  P_HOMO             68 non-null     float64
 14  P_LUMO             68 non-null     float64
 15  P_ESP_P            68 non-null     float64
 16  P_ESP_=O           68 non-nu

In [3]:
#Building descriptors
X = df.drop(columns=['Yield', 'Cate_name', 'P_name' ])
print('---Descriptors---')
print(X.head())

y = pd.DataFrame(df['Yield'],columns=['Yield'])
print('---Objective---')
print(y.head())

---Descriptors---
   Cate_HOMO  Cate_LUMO  Cate_ESP_O1  Cate_ESP_H1  Cate_ESP_O2  Cate_ESP_H2  \
0      -5.63       0.21    -0.555170     0.441278    -0.594799     0.444770   
1      -5.63       0.21    -0.555170     0.441278    -0.594799     0.444770   
2      -5.63       0.21    -0.555170     0.441278    -0.594799     0.444770   
3      -5.63       0.21    -0.555170     0.441278    -0.594799     0.444770   
4      -5.51       0.22    -0.542028     0.433177    -0.582447     0.440296   

   Cate_BL_ArO1  Cate_BL_O1H1  Cate_BL_ArO2  Cate_BL_O2H2  Cate_total_dipole  \
0         1.363         0.969         1.378         0.965           2.484773   
1         1.363         0.969         1.378         0.965           2.484773   
2         1.363         0.969         1.378         0.965           2.484773   
3         1.363         0.969         1.378         0.965           2.484773   
4         1.364         0.969         1.379         0.965           2.303134   

   P_HOMO  P_LUMO   P_ESP_

In [4]:
#storage 
data_r2_train =[]
data_RMSE_train = []
data_MAE_train = []
data_r2_test = []
data_RMSE_test = []
data_MAE_test = []
best_model_parametors = []
data_cv_score = []

#MLP
for i in range(10):
    seed=i
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)
    
    #autoscaling
    a_X_train = (X_train - X_train.mean(axis=0)) / X_train.std(axis=0, ddof=1)
    a_X_test = (X_test - X_train.mean(axis=0)) / X_train.std(axis=0, ddof=1)
    a_y_train = y_train/10
    
    param_mlp={'hidden_layer_sizes':[(128,),(256,),(512,)], 'alpha':[1,2,3,4,5]}
    reg_mlp = GridSearchCV(MLPRegressor(random_state=0, max_iter=1000, learning_rate_init=0.03),
                           param_grid=param_mlp, cv=5, n_jobs=16)
    reg_mlp.fit(a_X_train,a_y_train['Yield'])
    reg_best = reg_mlp.best_estimator_
    y_pred1 = reg_best.predict(a_X_train)*10
    y_pred2 = reg_best.predict(a_X_test)*10
            
    #train
    r2_train = metrics.r2_score(y_train, y_pred1)
    RMSE_train = metrics.mean_squared_error(y_train, y_pred1)
    MAE_train =  metrics.mean_absolute_error(y_train, y_pred1)
    
    #test
    r2_test = metrics.r2_score(y_test, y_pred2)
    RMSE_test = metrics.mean_squared_error(y_test, y_pred2)
    MAE_test = metrics.mean_absolute_error(y_test, y_pred2)
    
    #strage data of accuracy 
    parametors = reg_mlp.best_params_
    best_model_parametors.append(parametors)

    data_r2_train.append(r2_train)
    data_RMSE_train.append(RMSE_train)
    data_MAE_train.append(MAE_train)
    data_r2_test.append(r2_test)
    data_RMSE_test.append(RMSE_test)
    data_MAE_test.append(MAE_test)
    data_cv_score.append(reg_mlp.best_score_)
    
    print('----------------------')
    print('seed:', seed)
    print("Best Model Parameter:",reg_mlp.best_params_)
    print("Best Model Score:",reg_mlp.best_score_)
    print('R2_test:', r2_test)
print('R2_train_means:', sum(data_r2_train)/10)
print('CV_score_means:', sum(data_cv_score)/10)
print('R2_test_means:', sum(data_r2_test)/10)

----------------------
seed: 0
Best Model Parameter: {'alpha': 4, 'hidden_layer_sizes': (256,)}
Best Model Score: 0.735929356665834
R2_test: 0.7217158125469696
----------------------
seed: 1
Best Model Parameter: {'alpha': 2, 'hidden_layer_sizes': (256,)}
Best Model Score: 0.7493783081092319
R2_test: 0.6666125579939662
----------------------
seed: 2
Best Model Parameter: {'alpha': 5, 'hidden_layer_sizes': (256,)}
Best Model Score: 0.7630990786284133
R2_test: 0.7902452044930219
----------------------
seed: 3
Best Model Parameter: {'alpha': 2, 'hidden_layer_sizes': (256,)}
Best Model Score: 0.7443144123468546
R2_test: 0.8127274353031123
----------------------
seed: 4
Best Model Parameter: {'alpha': 5, 'hidden_layer_sizes': (512,)}
Best Model Score: 0.7449835819546037
R2_test: 0.7853020023337732
----------------------
seed: 5
Best Model Parameter: {'alpha': 5, 'hidden_layer_sizes': (256,)}
Best Model Score: 0.7494565194642778
R2_test: 0.7910602635602472
----------------------
seed: 6
Best

In [5]:
data_r2_train_pd = pd.DataFrame(data= data_r2_train, columns=['r2_train'])
data_RMSE_train_pd = pd.DataFrame(data=data_RMSE_train, columns=['RMSE_train'])
data_MAE_train_pd = pd.DataFrame(data=data_MAE_train, columns=['MAE_train'])
data_r2_test_pd = pd.DataFrame(data=data_r2_test, columns=['r2_test'])
data_RMSE_test_pd = pd.DataFrame(data=data_RMSE_test, columns=['RMSE_test'])
data_MAE_test_pd = pd.DataFrame(data=data_MAE_test, columns=['MAE_test'])
data_cv_score_pd = pd.DataFrame(data=data_cv_score, columns=['cv_score'])
data_parametors = pd.DataFrame([best_model_parametors])

data_all = pd.concat([data_r2_train_pd, data_RMSE_train_pd, data_MAE_train_pd, data_r2_test_pd, 
                      data_RMSE_test_pd, data_MAE_test_pd, data_cv_score_pd],
                     axis=1, join='inner')

data_all.loc['mean'] = data_all.mean()
print(data_all)

data_all.to_csv('../../score/DFT/MLP_score.csv')

      r2_train  RMSE_train  MAE_train   r2_test   RMSE_test   MAE_test  \
0     0.955176   33.047461   3.556768  0.721716  363.981835  14.054390   
1     0.985236   13.350828   2.324666  0.666613  323.001781  13.441460   
2     0.934003   61.053438   4.835025  0.790245  202.845729   9.038889   
3     0.976341   22.066601   2.878336  0.812727  179.734101   9.136724   
4     0.925996   63.798907   4.856959  0.785302  228.293590   9.780635   
5     0.891298   88.020500   5.225945  0.791060  254.113360  10.574546   
6     0.910334   73.586491   4.725225  0.784583  259.245512  11.047021   
7     0.936045   61.172953   5.122501  0.777377  201.809679   9.459016   
8     0.927289   63.993819   4.647438  0.838876  159.228848   9.079300   
9     0.926242   74.750035   5.544411  0.919205   56.933128   6.371843   
mean  0.936796   55.484103   4.371727  0.788770  222.918756  10.198382   

      cv_score  
0     0.735929  
1     0.749378  
2     0.763099  
3     0.744314  
4     0.744984  
5     0.7